In [1]:
%matplotlib inline

In [18]:
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [4]:
data_path = r'/home/alex/data/nlp/agmir/toy_data/trs_data'

In [30]:
from loaders import prepareReportData, normalizeString, tensorsFromPair

In [6]:
input_lang, output_lang, ds = prepareReportData()

read data...
read 3580 reports
trimmed to 3580 sentence pairs
counting words...
counted words:
	 tags 590
	 report 1953


# more processing

In [45]:
from loaders import EOS_token, SOS_token

In [8]:
MAX_LENGTH = 30

In [31]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])), input_lang, output_lang, device, MAX_LENGTH)

(tensor([[2],
         [1]], device='cuda:0'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94],
         [ 95],
         [ 25],
         [ 20],
         [ 21],
         [ 22],
         [175],
         [164],
         [  3],
         [ 41],
         [286],
         [ 25],
         [287],
         [  3],
         [  9],
         [ 27],
         [ 23],
         [ 29],
         [  3],
         [  9],
         [ 37]], device='cuda:0'))

# train

In [10]:
from models import EncoderRNN, AttnDecoderRNN

In [16]:
from train_utils import trainIters

In [46]:
%%time
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, max_length=MAX_LENGTH, dropout_p=0.1).to(device)

plot_losses = trainIters(ds, encoder1, attn_decoder1, 100, MAX_LENGTH, input_lang, output_lang, print_every=10)

0m 1s (- 0m 10s) (10 10%) 7.3962
0m 2s (- 0m 8s) (20 20%) 7.0021
0m 3s (- 0m 7s) (30 30%) 5.8903
0m 4s (- 0m 6s) (40 40%) 5.9442
0m 5s (- 0m 5s) (50 50%) 5.2607
0m 6s (- 0m 4s) (60 60%) 5.4074
0m 7s (- 0m 3s) (70 70%) 5.2488
0m 8s (- 0m 2s) (80 80%) 5.1021
0m 9s (- 0m 1s) (90 90%) 4.6294
0m 10s (- 0m 0s) (100 100%) 4.7985
CPU times: user 10.5 s, sys: 124 ms, total: 10.6 s
Wall time: 10.7 s


# WIP plot learning curve

In [47]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# eval

In [50]:
from perf import get_av_bleu
from test_utils import evaluateAll

In [ ]:
%%time
pred_output = evaluateAll(ds, encoder1, attn_decoder1, MAX_LENGTH, input_lang, output_lang)

In [ ]:
get_av_bleu(pred_output)